# Preliminary Data and Functions

# RNN Model

### Hyper Parameters

In [2]:
batchSize = 512
lstmUnits = 128
numClasses = 2
iterations = 10000
learning_rate = 0.001
maxSeqLength = 20

In [3]:
import tensorflow as tf
with tf.name_scope("Input_Data") as scope:
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name='Sentiment_Labels')
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name='Text')

In [4]:
with tf.name_scope("Data_Preprocessing") as scope:
    data = tf.Variable(tf.zeros([batchSize, maxSeqLength, 3]),dtype=tf.float32, name='Embed_Text')
    data = tf.nn.embedding_lookup(wordVectors,input_data)

In [5]:
with tf.name_scope("MultiRNN") as scope:
    lstmCell1 = tf.nn.rnn_cell.LSTMCell(lstmUnits)
    lstmCell1 = tf.nn.rnn_cell.DropoutWrapper(cell=lstmCell1, output_keep_prob=0.9)
    lstmCell2 = tf.nn.rnn_cell.LSTMCell(lstmUnits)
    lstmCell2 = tf.nn.rnn_cell.DropoutWrapper(cell=lstmCell2, output_keep_prob=0.9)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstmCell1, lstmCell2])
    value, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [6]:
with tf.name_scope("Prediction_Parameters") as scope:
    weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]), name='Weights')
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name='bias')
    value = tf.transpose(value, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
with tf.name_scope("Prediction") as scope:
    prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [7]:
with tf.name_scope("Accuracy") as scope:
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


In [8]:
with tf.name_scope("Loss") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
with tf.name_scope("Train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
import datetime
sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
validation = tf.summary.scalar('Validation', accuracy)
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [9]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


# Training

In [10]:
for i in range(iterations+1):
        #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
    #Write summary to Tensorboard
#    if (i % 5 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)
#        nextTestBatch, nextTestBatchLabels = getTestBatch();
#        val = sess.run(validation, {input_data: nextTestBatch, labels: nextTestBatchLabels})
#        writer.add_summary(val, i)
    if (i%500 == 0):
        print("Loss: " + str(sess.run(loss, {input_data: nextBatch, labels: nextBatchLabels})))
save_path = saver.save(sess, "RNN2/RNN2.ckpt", global_step=i)
writer.close()

Loss: 0.7169582
Loss: 0.5871757
Loss: 0.57802606
Loss: 0.5395138
Loss: 0.5586865
Loss: 0.5777421
Loss: 0.540998
Loss: 0.5669285
Loss: 0.5359461
Loss: 0.53072286
Loss: 0.54103416
Loss: 0.5577469
Loss: 0.52806693
Loss: 0.5096108
Loss: 0.50664335
Loss: 0.53114235
Loss: 0.5362724
Loss: 0.52452034
Loss: 0.5521227
Loss: 0.53186285
Loss: 0.5150936


NameError: name 'writer' is not defined

In [12]:
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix
def test(sent):
    x = ['negetive', 'positive']
    sent = getSentenceMatrix(sent)
    pred = sess.run(prediction, {input_data:sent})[0]
    return(pred)
def results():
    arr = np.array([[1, 0]], dtype=np.float32)
    arr_pred = np.array([test("my life sucks")])
    for i in range(512, ids.shape[0], 512):
        arr = np.vstack((arr, the_labels[i-512:i]))
        arr_pred = np.vstack((arr_pred, sess.run(prediction, {input_data:ids[i-512:i]})))
    x = np.argmax(arr_pred, axis=1)
    y = np.argmax(arr, axis=1)
    return((np.sum(np.equal(x, y))/x.shape))

In [13]:
results()

array([0.7748625])

In [14]:
def find_time():
    import time
    cx = np.random.randint(513, ids.shape[0])
    start_time = time.time()
    sess.run(prediction, {input_data:ids[cx-512:cx]})
#    print("---| %s seconds ---" % (time.time() - start_time))
    return(time.time() - start_time)
find_time()
def find():
    temp = 0
    for i in range(10):
        temp+=find_time()
    print(temp/10)
find()

0.24717836380004882
